In [ ]:
!pip install datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset
import seaborn as sns
import matplotlib.pyplot as plt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
df = pd.read_csv("final_stockcorpus_harsh.csv")
df = df[['Sentence', 'Sentiment']]


In [ ]:
df = df.dropna(subset=['Sentence', 'Sentiment'])
df['Sentiment'] = df['Sentiment'].astype(str)


In [ ]:
label_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
df['Sentiment'] = df['Sentiment'].map(label_map)

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Sentence'].tolist(), df['Sentiment'].tolist(), test_size=0.2, random_state=42
)

In [ ]:
train_dataset = Dataset.from_dict({'Sentence': train_texts, 'Sentiment': train_labels})
test_dataset = Dataset.from_dict({'Sentence': test_texts, 'Sentiment': test_labels})

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
     # Check if labels need mapping
    if isinstance(examples['Sentiment'][0], str):
        labels = [label_map[label] for label in examples['Sentiment']]
    else:
        labels = examples['Sentiment']

    # Fix the key name here
    tokenized = tokenizer(examples['Sentence'], padding=True, truncation=True, max_length=128)
    tokenized['labels'] = labels
    return tokenized

# Apply tokenization to the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Ensure that the dataset contains 'input_ids', 'attention_mask', and 'labels'
print(train_dataset[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/16435 [00:00<?, ? examples/s]

Map:   0%|          | 0/4109 [00:00<?, ? examples/s]

{'Sentence': 'this move made ibm a meaningful player in the hybrid cloud and after taking the ceo job in april 2020 krishna acquired more cloud companies', 'Sentiment': 2, 'input_ids': [101, 2023, 2693, 2081, 9980, 1037, 15902, 2447, 1999, 1996, 8893, 6112, 1998, 2044, 2635, 1996, 5766, 3105, 1999, 2258, 12609, 10871, 3734, 2062, 6112, 3316, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3).to("cuda")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./bert_sentiment_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    # Use DataCollatorWithPadding for dynamic padding
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)

In [ ]:
trainer.train()

In [ ]:
preds_output = trainer.predict(test_dataset)
preds = torch.tensor(preds_output.predictions).argmax(axis=1)

In [ ]:
print(preds_output)

In [ ]:
print(classification_report(test_labels, preds.tolist(), target_names=label_map.keys()))

In [ ]:
cm = confusion_matrix(test_labels, preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_map.keys(), yticklabels=label_map.keys())
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - BERT Sentiment Classifier")
plt.show()

In [ ]:
report = classification_report(test_labels, preds, target_names=label_map.keys(), output_dict=True)
df_report = pd.DataFrame(report).transpose()
print(df_report)


In [ ]:
df_report[:-1][['precision', 'recall', 'f1-score']].plot(kind='bar', figsize=(8,6))
plt.title("Precision, Recall & F1-Score per Class")
plt.ylabel("Score")
plt.ylim(0, 1)
plt.grid(axis='y')
plt.show()


In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score, roc_curve, auc

y_true = label_binarize(test_labels, classes=[0, 1, 2])
y_scores = preds_output.predictions

fpr = {}
tpr = {}
roc_auc = {}
for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

for i, label in enumerate(label_map.keys()):
    plt.plot(fpr[i], tpr[i], label=f'{label} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Multi-class ROC Curve")
plt.legend()
plt.grid()
plt.show()


In [ ]:
import numpy as np
wrong = np.where(preds != torch.tensor(test_labels))[0]

print("\nExamples of Misclassified Samples:")
for i in wrong[:5]:
    print(f"Text: {test_texts[i]}")
    true_label = list(label_map.keys())[test_labels[i]]
    pred_label = list(label_map.keys())[preds[i]]
    print(f"Actual: {true_label}, Predicted: {pred_label}\n")


In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to("cuda")
    outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    return list(label_map.keys())[pred]


print(predict_sentiment("The stock market is crashing badly."))
print(predict_sentiment("Everything is stable."))
print(predict_sentiment("Great performance!"))


In [ ]:
print(predict_sentiment("The Google had a great rise in shares."))

In [ ]:
  print(predict_sentiment("Tata has suffered a factory burn down"))

In [ ]:
print(predict_sentiment("Zomato has suffered a loss of 20%"))

In [ ]:
print(predict_sentiment("$400-million unemployment fraud under Biden-era? Elon Musk's DOGE drops another bombshell"))